Configure the following variables based on what you want to generate.

In [ ]:
temperature = 1.5 # I found that 1.5 is perfect for generating ideas, increase this for more chaos and
# creativity (beware of unintelligible generation). Decrease for more consistency
fields = ['Artificial Intelligence', 'Cognitive Psychology', 'Biological Computation',
          'Brain Machine Interface'] # modify this list depending on what fields you want to generate ideas

# Reference resources for inspiration, papers and articles (pdf and txt format)
# Don't forget to add them in Files, on the left side of the screen with drag and drop
# Website can be saved as PDF by pressing Print and then save pdf
resources = ['entropy-26-00481.pdf',
             'Personality-changes.pdf',
             'The future of mind.pdf']

chunks_limit = 3 # limit on how many chunks we summarize per article, increase
# this if you want the LLM to get more inspiration from your articles

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = 8192,
    load_in_4bit = True
)

In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
    return text

def chunk_text(text, max_chars=2000):
    chunks = []
    current_pos = 0
    while current_pos < len(text):
        end_pos = min(current_pos + max_chars, len(text))
        chunks.append(text[current_pos:end_pos])
        current_pos = end_pos
    return chunks

def summarize_text_in_idea(chunk):
    messages = [
        {"from": "system", "value": "You are a creative summarizer tool for concise and interesting ideas."},
        {"from": "human", "value": f"""
        Your task is to concisely summarize the following fragment into one or multiple ideas:
        {chunk}
         Please ignore any metadata such as description about the published articles,
         author details and more. Do not provide it as a number list, just plain text \
         delimited by new line.
        """}
    ]

    # Create the input IDs (no streamer)
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    prompt_len = inputs.shape[-1]

    # Generate tokens
    output_ids = model.generate(
        input_ids=inputs,
        max_new_tokens=1024,
        use_cache=True,
        temperature=0.7
    )

    new_tokens = output_ids[0][prompt_len:]

    # Decode the tokens to text
    generated_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return generated_text

def determine_context(resources):
  extracted_ideas = []
  for pdf_path in resources:
    text = extract_text_from_pdf(pdf_path)
    chunked_text = chunk_text(text)[:chunks_limit] # we just take the first X chunks to make
    # it run faster

    for chunk in chunked_text:
      curr_idea = summarize_text_in_idea(chunk)
      extracted_ideas.append(curr_idea)

  return "\n".join(extracted_ideas)

In [ ]:
context = determine_context(resources)

In [ ]:
print('The following ideas are generated from the articles and will be used for context:')
print(context)

In [ ]:
if len(fields) > 1:
    concat_fields = ', '.join(fields[:-1]) + ', and ' + fields[-1]
else:
    concat_fields = fields[0]

# system_prompt = f"""
# You are a creative scientist specialized in coming up with groundbreaking, creative \
# ideas in all fields. You have a history of ideas that came to life related to (and \
# not only) psychology, neuroscience, physics, biology, arts, history, artficial intelligence \
# biology.
# """

system_prompt = f"""
You are a creative scientist specialized in purely speculative, theoretical ideas.
Below is an example of the style we want:

Example style:
'Imagine a theoretical framework where AI consciousness arises spontaneously from
self-reflection. The system would be more of a philosophical or abstract construct
rather than an engineered project...'

You are used to generate multiple ideas similar to this.

End of example.
"""

generator_prompt = f"""
Generate bold, unconventional, and speculative ideas that merge concepts from \
{concat_fields}. Focus on challenging existing assumptions, creating disruptive \
innovations, or proposing entirely new frameworks. Emphasize creativity and \
originality, prioritizing novelty over practicality. Identify emerging, \
underexplored problems or challenges that could benefit from innovative solutions, \
and suggest how your ideas could address them. Not only suggest the idea but also \
provide a framework of how to actually achieve it.

Use the following extracted fragments as resources to guide your ideas:
{context}
"""

messages = [
    {"from": "system", "value": system_prompt},
    {"from": "human", "value": generator_prompt}
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

# Count how many tokens in the prompt
prompt_len = inputs.shape[-1]

text_streamer = TextStreamer(tokenizer)
gen_tokens = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True, temperature=temperature)

print(tokenizer.batch_decode(gen_tokens[:, prompt_len:])[0])